<center><h1 style="font-size: 40px"> Дообученная модель ruBert для задачи экстрактной суммаризации текстов.</h1></center>

# Содержание
### 1. [Дообучение ruBert](#chapter1)
#### 1.1. [Загрузка модели](#chapter1.1)
#### 1.2. [Загрузка и предобработка датасета](#chapter1.2)
#### 1.3. [Дообучение модели ruBert](#chapter1.3)
#### 1.4. [Тестирование дообученной модели](#chapter1.4)

### 2. [Суммаризация текста](#chapter2)
#### 2.1. [Использование библиотеки summarizer](#chapter2.1)
#### 2.2. [Подсчёт всех метрик](#chapter2.2)

### 3. [Вывод](#chapter3)

In [1]:
import numpy as np
import pandas as pd

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
from transformers import BertModel, BertForSequenceClassification, BertTokenizer, AutoConfig

In [4]:
import warnings
warnings.filterwarnings("ignore")

<center id="chapter1"><h1 style="font-size: 24px"> 1. Дообучение ruBert </h1></center>

## 1.1. Загрузка модели <a id="chapter1.1"></a>

In [7]:
model_name = 'DeepPavlov/rubert-base-cased'

tokenizer = BertTokenizer.from_pretrained(model_name)

custom_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2, output_hidden_states=True).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
custom_model

In [ ]:
# Зафиксируем веса базовой модели, чтобы при обучении их не трогать и обучать только последний слой (классификатор)
for param in custom_model.base_model.parameters():
    param.requires_grad = False

## 1.2. Получение и предобработка данных <a id="chapter1.2"></a>

In [ ]:
# Исходный датасет для задачи экстрактной суммаризации
dataset = pd.read_pickle("../Data/ru_train.pkl")
dataset

In [ ]:
def make_classification_df(dataset):
    """
    Преобразует исходный датасет в формат, подходящий для решения задачи бинарной классификации.
    """
    train_df = pd.DataFrame()
    
    sentences = []
    labels = []

    for i in dataset.index:
        sentences += list(dataset.loc[i]["sentences"])
        labels += list(dataset.loc[i]["labels"])

    train_df["sentence"] = sentences
    train_df["label"] = labels

    return train_df

In [ ]:
# Создадим датасет бинарной классификации на части текстов для начала
df = make_classification_df(dataset.loc[0:99])
df

In [ ]:
# Пусть обучающая часть содержит 80% предложений, а в валидационной и тестовой оставим по 10%
df_train = df.loc[0:int(len(df)*0.8)]
df_val = df.loc[int(len(df)*0.8)+1:int(len(df)*0.9)]
df_test = df.loc[int(len(df)*0.9):]

In [ ]:
display(df_train)
display(df_val)
display(df_test)

In [ ]:
class TextDataset(Dataset):
    """Класс для преобразования датасета к нужному формату"""
    def __init__(self, dataframe, tokenizer, max_len=64):
        self.tokenizer = tokenizer
        self.sentences = dataframe['sentence'].tolist()
        self.labels = dataframe['label'].tolist()
        self.max_len = max_len

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        text = self.sentences[idx]
        label = self.labels[idx]
        # токенизируем
        inputs = self.tokenizer.encode_plus(
            text=text, # наши данные
            text_pair=None, # это для задачи вопросно-ответной системы, т.е. не для нас
            add_special_tokens=True, # добавление спец-токенов, отвечающих за "начало предложения" [CLS] и "конец предложения" [SEP]
            max_length=self.max_len, # максимальная длина последовательности
            padding='max_length', # если в предложении меньше 64 токенов, то остальные заменяем на пустые
            truncation=True, # если в предложениее 64+ токенов, то мы просто обрезаем их
            return_token_type_ids=False, # это для задачи вопросно-ответной системы, т.е. не для нас
            return_attention_mask=True, # это говорит нашей модели, какие токены важны, а какие просто как padding или [CLS] и т.д.
            return_tensors='pt' # формат выдачи токенизатора, в нашем случае - torch тензор
        )

        # то что мы запихнем в модель
        return {
            'input_ids': inputs['input_ids'].flatten(), # это наши цифровые токены (т.е. для токена 'привет' будет какое-нибудь '105')
            'attention_mask': inputs['attention_mask'].flatten(), # это наши маски
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
train_dataset = TextDataset(df_train, tokenizer)
val_dataset = TextDataset(df_val, tokenizer)
test_dataset = TextDataset(df_test, tokenizer)

batch_size = 16

train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## 1.3. Дообучение ruBert <a id="chapter1.3"></a>

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
import tqdm
from tqdm.auto import tqdm

def train_val(model, train_data_loader, val_data_loader, loss_fn, optimizer, device, num_epochs):
    for t in tqdm(range(num_epochs)):
        train_epoch_loss = []
        eval_epoch_loss = []

        # Будем сохранять правильные метки и предсказания для расчёта метрики f1
        train_labels = []
        train_predictions = []

        eval_labels = []
        eval_predictions = []
        
        print("Эпоха номер: ", t)

        # Обучение модели на текущей эпохе
        model.train()
        for train_data in tqdm(train_data_loader):
            input_ids = train_data['input_ids'].to(device) # токены
            attention_mask = train_data['attention_mask'].to(device) # маски
            labels = train_data['labels'].to(device) # класс

            train_labels += list(labels)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask) # результат модели
            _, preds = torch.max(outputs.logits, dim=1)

            train_predictions += list(preds)
            
            loss = loss_fn(outputs.logits, labels) # считаем потерю
            train_epoch_loss.append(loss.item())


            # Выполним подсчёт новых градиентов
            loss.backward()
            # Выполним шаг градиентного спуска
            optimizer.step()
            # Обнулим сохраненные у оптимизатора значения градиентов
            # перед следующим шагом обучения
            optimizer.zero_grad()

        # Оценка модели на валидационных данных после обучения на текущей эпохе
        model.eval()
        for val_data in tqdm(val_data_loader):
            input_ids = val_data['input_ids'].to(device) # токены
            attention_mask = val_data['attention_mask'].to(device) # маски
            labels = val_data['labels'].to(device) # класс

            eval_labels += list(labels)

            with torch.no_grad():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask) # результат модели
                _, preds = torch.max(outputs.logits, dim=1)

                eval_predictions += list(preds)
            
                loss = loss_fn(outputs.logits, labels) # считаем потерю
                eval_epoch_loss.append(loss.item())

        # Выведем результаты прошедшей эпохи обучения
        print("Train loss: ", np.mean(train_epoch_loss))
        print("Eval loss: ", np.mean(eval_epoch_loss))
        print("Train F1-score: ", f1_score(train_labels, train_predictions))
        print("Eval F1-score: ", f1_score(eval_labels, eval_predictions))
        
    return model

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss().to(device)

from transformers import AdamW
optimizer = AdamW(custom_model.parameters(), lr=1e-5)

num_epochs = 2

In [ ]:
custom_model = train_val(custom_model, train_data_loader, val_data_loader, loss_fn, optimizer, device, num_epochs)

In [ ]:
torch.save(custom_model.state_dict(), "custom_rubert_clf.pth")

## 1.4. Тестирование дообученной модели <a id="chapter1.4"></a>

In [ ]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2, output_hidden_states=True).to(device)
model.load_state_dict(torch.load("custom_rubert_clf.pth"))

In [ ]:
def test_predict(model, test_data_loader):
    test_labels = []
    test_predictions = [] 
    
    for test_data in tqdm(test_data_loader):
        input_ids = test_data['input_ids'].to(device) # токены
        attention_mask = test_data['attention_mask'].to(device) # маски
        labels = test_data['labels'].to(device) # класс

        test_labels += labels.tolist()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask) # результат модели
        _, preds = torch.max(outputs.logits, dim=1)

        test_predictions += preds.tolist()

    print(f1_score(test_labels, test_predictions))
    
    return test_predictions

In [ ]:
with torch.no_grad():
    df_test["predictions"] = test_predict(custom_model, test_data_loader)

<center id="chapter2"><h1 style="font-size: 24px"> 2. Суммаризация текста </h1></center>

Наш датасет:

In [5]:
extsum_dataset = pd.read_pickle("../Data/ru_test_5k.pkl")
extsum_dataset = pd.DataFrame(extsum_dataset[['ru_src', 'labels']].values, columns=['sentences', 'labels'])
# Взял небольшое количество для проверки кода
extsum_dataset_sample = extsum_dataset.loc[0:10]
extsum_dataset_sample

,sentences,labels
0,[(CNN) Палестинская администрация официально с...,"[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[(CNN) Не говоря уже о кошках, у которых девят...","[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,[(CNN) Если вы в последнее время следили за но...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[(CNN) Пятеро американцев, за которыми в течен...","[0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0]"
4,"[(CNN) Студент Дьюка признался, что повесил пе...","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
5,[(CNN) Он первоклассный баскетбольный новобран...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
6,[(CNN) Правительства во всем мире используют у...,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,"[(CNN) Эндрю Гетти, один из наследников миллиа...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, ..."
8,"[(CNN) Филиппинцев предупреждают, чтобы они бы...","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
9,[(CNN) Впервые за восемь лет легенда телевиден...,"[0, 0, 1, 1, 0, 0]"


Наша кастомная модель:

In [8]:
custom_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2, output_hidden_states=True).to(device)
custom_model.load_state_dict(torch.load("custom_rubert_clf.pth"))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

## 2.1. Использование библиотеки summarizer <a id="chapter2.1"></a>

In [9]:
from summarizer import Summarizer

In [10]:
bertsum_custom_model = Summarizer(custom_model=custom_model, custom_tokenizer=tokenizer)

In [11]:
text = " ".join(extsum_dataset_sample.loc[0]["sentences"])
text

'(CNN) Палестинская администрация официально стала 123-м членом Международного уголовного суда в среду, шаг, который дает суду юрисдикцию над предполагаемыми преступлениями на палестинских территориях. Официальное вступление было отмечено церемонией в Гааге (Нидерланды), где находится суд. Палестинцы подписали основополагающий Римский статут МУС в январе, когда они также признали его юрисдикцию в отношении предполагаемых преступлений, совершенных «на оккупированной палестинской территории, включая Восточный Иерусалим, с 13 июня 2014 года». Позже в том же месяце МУС начал предварительное расследование ситуации на палестинских территориях, открыв путь для возможного расследования военных преступлений против израильтян. Палестинцам, как членам суда, также могут быть предъявлены встречные обвинения. Израиль и США, ни одна из которых не является членом МУС, выступили против попыток палестинцев присоединиться к этому органу. Но министр иностранных дел Палестины Риад аль-Малики, выступая на ц

In [12]:
bertsum_custom_model(text, num_sentences=3)

'(CNN) Палестинская администрация официально стала 123-м членом Международного уголовного суда в среду, шаг, который дает суду юрисдикцию над предполагаемыми преступлениями на палестинских территориях. Правительства, стремящиеся наказать Палестину за присоединение к МУС, должны немедленно прекратить свое давление, а страны, которые поддерживают всеобщее признание договора о суде, должны высказаться и приветствовать ее членство», - сказал Балкис Джарра, советник по международному правосудию группы. « Международный уголовный суд был создан в 2002 году для преследования геноцида, преступлений против человечности и военных преступлений.'

## 2.2. Подсчёт всех метрик <a id="chapter2.2"></a>

Наши предыдущие результаты:

In [13]:
final_results = pd.read_csv("final_results.csv", index_col=0)
final_results

,rouge1,rouge2,rougeL,rougeLsum,bleu,meteor,bert_precision,bert_recall,bert_f1
TextRank,0.211472,0.106501,0.206855,0.206569,0.178233,0.312205,0.732481,0.754939,0.743080
LexRank,0.235729,0.118226,0.230788,0.230648,0.261628,0.322667,0.736673,0.737441,0.736529
LSA,0.173570,0.078269,0.170758,0.170495,0.175943,0.269465,0.704227,0.721596,0.712457
KL,0.203493,0.100092,0.200029,0.200301,0.191725,0.295647,0.712154,0.728635,0.719865
Luhn,0.228917,0.113189,0.224979,0.224915,0.230286,0.346023,0.727853,0.742424,0.734508
BertSum,0.245710,0.116171,0.241740,0.242022,0.242611,0.374681,0.732481,0.754939,0.743080


Наши метрики:

In [14]:
import evaluate

In [15]:
rouge = evaluate.load("rouge")
bleu  = evaluate.load("bleu")
meteor = evaluate.load('meteor')

bertscore  = evaluate.load("bertscore")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Максат\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Максат\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Максат\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [24]:
def get_metrics(generated_summaries, reference_summaries):
    
    metrics_dict = rouge.compute(predictions=generated_summaries, references=reference_summaries)
    metrics_dict["bleu"] = bleu.compute(predictions=generated_summaries, references=reference_summaries)["bleu"]
    metrics_dict["meteor"] = meteor.compute(predictions=generated_summaries, references=reference_summaries)["meteor"]

    bertscore_results = bertscore.compute(predictions=generated_summaries, references=reference_summaries, lang='ru')
    
    bert_dict = {'bert_precision': np.mean(bertscore_results['precision']), 
                 'bert_recall': np.mean(bertscore_results['recall']),
                'bert_f1': np.mean(bertscore_results['f1'])}
    
    return metrics_dict | bert_dict

In [25]:
def get_reference_summary(sentences, labels):
    """ 
    Получение образцовой экстрактивной суммаризации по массиву предложений и по маске входящих в суммаризацию предложений
    Параметры:
        sentences - массив строк(предложений)
        labels - маска предложений, 1 - предложение входит в суммаризацию, 0 - не входит
    Возвращает:
        строку
    """
    labels = [x==1 for x in labels]
    
    reference_summary = np.array(sentences)[labels]

    return ' '.join(reference_summary)

In [26]:
# В этом датафрейме будет два столбца:
# 1. References - эталонные суцммаризации
# 2. CustomBertSum - сгенерированные суммаризации 
df_results = pd.DataFrame()

# Заполняем столбец References
df_results["References"] = extsum_dataset_sample[['sentences', 'labels']].apply(lambda cols: get_reference_summary(cols['sentences'], cols['labels']) , axis=1)
df_results

,References
0,Позже в том же месяце МУС начал предварительно...
1,"Ее взяла к себе жительница Мозес-Лейк, штат Ва..."
2,"Он, конечно, министр иностранных дел Ирана. В ..."
3,"В марте они заразились Эболой в Сьерра-Леоне, ..."
4,"(CNN) Студент Дьюка признался, что повесил пет..."
5,Она первокурсница средней школы с синдромом Да...
6,(CNN) Правительства во всем мире используют уг...
7,"Родители Гетти, Энн и Гордон Гетти, опубликова..."
8,Всего несколько дней назад Майсак получил стат...
9,в выпуске «The Price Is Right» от 1 апреля я в...


In [27]:
# Суммаризируем все тексты
df_results['CustomBertSum'] = extsum_dataset_sample[['sentences', 'labels']].apply(lambda cols: bertsum_custom_model(' '.join(cols['sentences']), num_sentences=sum(cols['labels'])),
                                                          axis=1)

In [28]:
# Cохраним на всякий полученные суммаризации
df_results.to_pickle("custom_bert_summarizations.pkl")

In [29]:
# Посчитаем все метрики
metrics_dict = get_metrics(df_results['CustomBertSum'].values, df_results["References"].values)
metrics_dict

{'rouge1': 0.2121212121212121,
 'rouge2': 0.14285714285714288,
 'rougeL': 0.2121212121212121,
 'rougeLsum': 0.2121212121212121,
 'bleu': 0.24579650073762876,
 'meteor': 0.37461433231653807,
 'bert_precision': 0.7227218801325018,
 'bert_recall': 0.7554647109725259,
 'bert_f1': 0.7385479536923495}

In [30]:
final_results.loc["Custom Bert"] = metrics_dict
final_results

,rouge1,rouge2,rougeL,rougeLsum,bleu,meteor,bert_precision,bert_recall,bert_f1
TextRank,0.211472,0.106501,0.206855,0.206569,0.178233,0.312205,0.732481,0.754939,0.743080
LexRank,0.235729,0.118226,0.230788,0.230648,0.261628,0.322667,0.736673,0.737441,0.736529
LSA,0.173570,0.078269,0.170758,0.170495,0.175943,0.269465,0.704227,0.721596,0.712457
KL,0.203493,0.100092,0.200029,0.200301,0.191725,0.295647,0.712154,0.728635,0.719865
Luhn,0.228917,0.113189,0.224979,0.224915,0.230286,0.346023,0.727853,0.742424,0.734508
BertSum,0.245710,0.116171,0.241740,0.242022,0.242611,0.374681,0.732481,0.754939,0.743080
Custom Bert,0.212121,0.142857,0.212121,0.212121,0.245797,0.374614,0.722722,0.755465,0.738548


<center id="chapter3"><h1 style="font-size: 24px"> 3. Вывод </h1></center>

Окончательный выбор лучшей модели! Ура!